In [25]:
import requests, zipfile, io
import pandas as pd
import numpy as np
import bokeh.plotting as bpl
from datetime import datetime as dt
from bokeh.io import export_png

from pathlib import Path

In [53]:
DATOS_BRUTOS = Path("../datos/brutos/")
DATOS_PROCESADOS = Path("../datos/procesados/")
FIGURAS = Path("../reportes/figuras/")

In [2]:
bpl.output_notebook()

Loading BokehJS ...

Adquisición de datos históricos, esta parte solo es necesaria una vez, guarda los archivos a disco

Carga de datos históricos y actuales y conversión a DataFrame, para cada archivo se agrega una columna al dataframe que hace referencia a la fecha de la publicación del archivo, el país va a estar roto por problemas de encoding. En libre office se pueden abrir como utf8 pero python detecta un caracter que no puede decodificar y no abre los archivos. Se están forzando a 'latin1' pero entonces los acentos no son correctos.

Para descargar los datos corre el _script_ `obtener_datos.py` que se encuentra en la carpeta `src/datos`

desde tu terminal
```bash
python -m src/datos/obtener_datos.py
```

In [37]:
frames = []
for archivo in DATOS_BRUTOS.glob("*COVID19MEXICO.csv"):
    data = pd.read_csv(archivo, encoding="latin1", low_memory = False,)
    data["FECHA_ARCHIVO"] = dt.strptime(archivo.name[:6],"%y%m%d").date()
    frames.append(data)

In [38]:
len(frames)

44

In [39]:
frames[0].head()

,FECHA_ACTUALIZACION,ID_REGISTRO,ORIGEN,SECTOR,ENTIDAD_UM,SEXO,ENTIDAD_NAC,ENTIDAD_RES,MUNICIPIO_RES,TIPO_PACIENTE,...,OBESIDAD,RENAL_CRONICA,TABAQUISMO,OTRO_CASO,RESULTADO,MIGRANTE,PAIS_NACIONALIDAD,PAIS_ORIGEN,UCI,FECHA_ARCHIVO
0,2020-04-19,09e8dc,2,9,15,1,15,15,37.0,2,...,2,2,2,2,1,99,99,97,1,2020-04-13
1,2020-04-19,0efbaf,2,9,28,2,16,28,32.0,1,...,2,2,2,1,1,99,99,97,97,2020-04-13
2,2020-04-19,091a48,1,12,15,2,15,15,31.0,2,...,2,2,2,2,2,99,99,97,2,2020-04-13
3,2020-04-19,13236c,1,4,2,2,21,2,4.0,2,...,1,2,2,99,2,99,MÃ©xico,97,2,2020-04-13
4,2020-04-19,0cdb12,1,12,9,2,99,9,12.0,2,...,2,2,2,2,1,99,MÃ©xico,97,2,2020-04-13


In [40]:
all_data = pd.concat(frames,axis=0,ignore_index=True)

Conversión de fechas a formato de fecha

In [41]:
variables_fechas = [col for col in all_data.columns if "FECHA_" in col]

In [42]:
all_data[variables_fechas]

,FECHA_ACTUALIZACION,FECHA_INGRESO,FECHA_SINTOMAS,FECHA_DEF,FECHA_ARCHIVO
0,2020-04-19,2020-04-09,2020-03-28,9999-99-99,2020-04-13
1,2020-04-19,2020-04-06,2020-04-04,9999-99-99,2020-04-13
2,2020-04-19,2020-04-06,2020-04-04,9999-99-99,2020-04-13
3,2020-04-19,2020-04-02,2020-03-26,9999-99-99,2020-04-13
4,2020-04-19,2020-03-28,2020-03-17,9999-99-99,2020-04-13
...,...,...,...,...,...
4898598,2020-05-18,2020-05-13,2020-05-01,9999-99-99,2020-05-18
4898599,2020-05-18,2020-05-12,2020-05-11,9999-99-99,2020-05-18
4898600,2020-05-18,2020-04-16,2020-04-16,9999-99-99,2020-05-18
4898601,2020-05-18,2020-05-13,2020-05-10,9999-99-99,2020-05-18


In [36]:
for variable in variables_fechas:
    all_data[variable] = pd.to_datetime(all_data[variable], format = "%Y-%m-%d", errors='coerce')

In [29]:
all_data["FECHA_ARCHIVO"] = pd.to_datetime(all_data["FECHA_ARCHIVO"],format="%Y-%m-%d")
all_data["FECHA_ACTUALIZACION"] = pd.to_datetime(all_data["FECHA_ACTUALIZACION"],format="%Y-%m-%d")
all_data["FECHA_INGRESO"] = pd.to_datetime(all_data["FECHA_INGRESO"],format="%Y-%m-%d")
all_data["FECHA_DEF"] = pd.to_datetime(all_data["FECHA_DEF"].replace({"9999-99-99":None}),format="%Y-%m-%d")
all_data["FECHA_SINTOMAS"] = pd.to_datetime(all_data["FECHA_SINTOMAS"],format="%Y-%m-%d")

In [43]:
all_data.columns

Index(['FECHA_ACTUALIZACION', 'ID_REGISTRO', 'ORIGEN', 'SECTOR', 'ENTIDAD_UM',
       'SEXO', 'ENTIDAD_NAC', 'ENTIDAD_RES', 'MUNICIPIO_RES', 'TIPO_PACIENTE',
       'FECHA_INGRESO', 'FECHA_SINTOMAS', 'FECHA_DEF', 'INTUBADO', 'NEUMONIA',
       'EDAD', 'NACIONALIDAD', 'EMBARAZO', 'HABLA_LENGUA_INDIG', 'DIABETES',
       'EPOC', 'ASMA', 'INMUSUPR', 'HIPERTENSION', 'OTRA_COM',
       'CARDIOVASCULAR', 'OBESIDAD', 'RENAL_CRONICA', 'TABAQUISMO',
       'OTRO_CASO', 'RESULTADO', 'MIGRANTE', 'PAIS_NACIONALIDAD',
       'PAIS_ORIGEN', 'UCI', 'FECHA_ARCHIVO'],
      dtype='object')

Agrupación de registros por id único

In [44]:
grupos_id = all_data.groupby("ID_REGISTRO")

Definición de dataframe por pacientes con fechas relevantes a su historia

In [45]:
pacientes = grupos_id[["FECHA_SINTOMAS","FECHA_ARCHIVO","FECHA_INGRESO","FECHA_DEF"]].min()

Definición de función para extracción de fecha de primer evento en la historia de un paciente

In [46]:
def fechador(frame,columna,valor):
    sframe = frame.sort_values("FECHA_ARCHIVO")
    fframe = sframe[sframe[columna]==valor]
    if(len(fframe)==0):
        return None
    ind_s = sframe.index[0]
    ind_f = fframe.index[0]
    if(ind_s==ind_f):
        return sframe["FECHA_INGRESO"].min()
    else:
        return fframe.loc[ind_f,"FECHA_ARCHIVO"]

Obtención de fechas de primer evento para todos los pacientes para algunos eventos importantes, esta parte del código tarda bastante y no está optimizada

In [47]:
fec_pos = grupos_id.apply(lambda x: fechador(x,"RESULTADO",1))
fec_neg = grupos_id.apply(lambda x: fechador(x,"RESULTADO",2))
fec_hos = grupos_id.apply(lambda x: fechador(x,"TIPO_PACIENTE",1))
fec_amb = grupos_id.apply(lambda x: fechador(x,"TIPO_PACIENTE",2))
fec_int = grupos_id.apply(lambda x: fechador(x,"INTUBADO",1))
fec_noint = grupos_id.apply(lambda x: fechador(x,"INTUBADO",2))
fec_noa_int = grupos_id.apply(lambda x: fechador(x,"INTUBADO",97))
fec_ign_int = grupos_id.apply(lambda x: fechador(x,"INTUBADO",99))
fec_neu = grupos_id.apply(lambda x: fechador(x,"NEUMONIA",1))
fec_noneu = grupos_id.apply(lambda x: fechador(x,"NEUMONIA",2))
fec_ign_neu = grupos_id.apply(lambda x: fechador(x,"NEUMONIA",99))
fec_uci = grupos_id.apply(lambda x: fechador(x,"UCI",1))
fec_nouci = grupos_id.apply(lambda x: fechador(x,"UCI",2))
fec_noa_uci = grupos_id.apply(lambda x: fechador(x,"UCI",97))
fec_ign_uci = grupos_id.apply(lambda x: fechador(x,"UCI",99))

In [48]:
pacientes["FECHA_POSITIVO"] = fec_pos
pacientes["FECHA_NEGATIVO"] = fec_neg
pacientes["FECHA_HOSPITALIZACION"] = fec_hos
pacientes["FECHA_AMBULATORIO"] = fec_amb
pacientes["FECHA_INTUBACION"] = fec_int
pacientes["FECHA_NO_INTUBACION"] = fec_noint
pacientes["FECHA_NO_APL_INTUBACION"] = fec_noa_int
pacientes["FECHA_SE_IGN_INTUBACION"] = fec_ign_int
pacientes["FECHA_NEUMONIA"] = fec_neu
pacientes["FECHA_NO_NEUMONIA"] = fec_noneu
pacientes["FECHA_SE_IGN_NEUMONIA"] = fec_ign_neu
pacientes["FECHA_UCI"] = fec_uci
pacientes["FECHA_NO_UCI"] = fec_nouci
pacientes["FECHA_NOAPL_UCI"] = fec_noa_uci
pacientes["FECHA_SE_IGN_UCI"] = fec_ign_uci

In [49]:
pacientes.rename(columns={"FECHA_ARCHIVO":"FECHA_APARICION"},inplace=True)

Exportar las historias por paciente a un archivo

In [51]:
fecha_file = dt.today().strftime("%d_%m_%Y")

In [54]:
pacientes.to_csv(DATOS_PROCESADOS / f"historia_pacientes_{fecha_file}.csv")

Búsqueda de pacientes en específico con alguna condición

In [55]:
columna = "FECHA_UCI"
pacientes[(pacientes[columna].notna())&(pacientes["FECHA_INGRESO"]!=pacientes[columna])]

,FECHA_SINTOMAS,FECHA_APARICION,FECHA_INGRESO,FECHA_DEF,FECHA_POSITIVO,FECHA_NEGATIVO,FECHA_HOSPITALIZACION,FECHA_AMBULATORIO,FECHA_INTUBACION,FECHA_NO_INTUBACION,FECHA_NO_APL_INTUBACION,FECHA_SE_IGN_INTUBACION,FECHA_NEUMONIA,FECHA_NO_NEUMONIA,FECHA_SE_IGN_NEUMONIA,FECHA_UCI,FECHA_NO_UCI,FECHA_NOAPL_UCI,FECHA_SE_IGN_UCI
ID_REGISTRO,,,,,,,,,,,,,,,,,,,
000f0f,2020-04-27,2020-05-02,2020-05-02,2020-05-03,2020-05-03,None,None,2020-05-02,2020-05-20,2020-05-02,None,None,2020-05-20,2020-05-02,None,2020-05-20,2020-05-02,None,None
0034c6,2020-04-27,2020-05-02,2020-04-27,9999-99-99,None,2020-05-04,None,2020-04-27,None,2020-04-27,None,None,2020-04-27,2020-05-12,None,2020-05-19,2020-04-27,None,None
004593,2020-04-27,2020-05-02,2020-05-01,2020-05-19,2020-05-03,None,None,2020-05-01,None,2020-05-01,None,None,2020-05-20,2020-05-01,None,2020-05-20,2020-05-01,None,None
00481b,2020-04-13,2020-04-15,2020-04-15,2020-04-22,2020-04-17,None,None,2020-04-15,2020-05-02,2020-04-15,None,None,2020-04-15,None,None,2020-05-02,2020-04-15,None,None
004b54,2020-04-25,2020-04-28,2020-04-27,2020-05-06,2020-05-03,None,None,2020-04-27,2020-05-07,2020-04-27,None,None,2020-04-27,None,None,2020-05-07,2020-04-27,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1e7378,2020-05-05,2020-05-07,2020-05-06,2020-05-07,2020-05-08,None,None,2020-05-06,2020-05-08,2020-05-06,None,None,2020-05-06,None,None,2020-05-08,2020-05-06,None,None
1e7968,2020-03-28,2020-04-12,2020-03-30,2020-04-06,2020-03-30,None,None,2020-03-30,2020-04-14,2020-03-30,None,None,2020-04-14,2020-03-30,None,2020-04-14,2020-03-30,None,None
1e7a63,2020-03-24,2020-04-12,2020-03-24,2020-04-02,None,2020-03-24,None,2020-03-24,None,2020-03-24,None,None,2020-03-24,None,None,2020-05-10,2020-03-24,None,None


Graficación de la historia de un paciente

In [56]:
caso_tmp = all_data[all_data["ID_REGISTRO"]=="00b7d8"][["ID_REGISTRO","RESULTADO","TIPO_PACIENTE","FECHA_ARCHIVO","FECHA_SINTOMAS","FECHA_INGRESO","FECHA_DEF","INTUBADO","NEUMONIA","UCI"]]
caso_tmp

,ID_REGISTRO,RESULTADO,TIPO_PACIENTE,FECHA_ARCHIVO,FECHA_SINTOMAS,FECHA_INGRESO,FECHA_DEF,INTUBADO,NEUMONIA,UCI
66665,00b7d8,1,2,2020-05-14,2020-04-12,2020-04-17,2020-04-25,1,1,1
234623,00b7d8,1,2,2020-04-26,2020-04-12,2020-04-17,9999-99-99,2,1,2
301467,00b7d8,3,2,2020-04-18,2020-04-12,2020-04-17,9999-99-99,2,1,2
343005,00b7d8,1,2,2020-05-21,2020-04-12,2020-04-17,2020-04-25,1,1,1
519625,00b7d8,1,2,2020-05-04,2020-04-12,2020-04-17,2020-04-25,1,1,1
646534,00b7d8,1,2,2020-05-25,2020-04-12,2020-04-17,2020-04-25,1,1,1
877309,00b7d8,1,2,2020-04-22,2020-04-12,2020-04-17,9999-99-99,2,1,2
899592,00b7d8,1,2,2020-05-10,2020-04-12,2020-04-17,2020-04-25,1,1,1
1121018,00b7d8,1,2,2020-04-29,2020-04-12,2020-04-17,9999-99-99,2,1,2
1221045,00b7d8,1,2,2020-05-15,2020-04-12,2020-04-17,2020-04-25,1,1,1


In [57]:
no_positivo, positivo, pendiente = "#00BFA5","#DD5246","#F8C600"
ambulatorio, hospitalizado, no_esp = "#FF9633", "#E10C0D", "#D8DBDD"
si, no, no_aplica, se_ignora = "#970A03", "#1F7249", "#E0E0E0", "#A6A6A6"

In [59]:
registro = caso_tmp["ID_REGISTRO"].min()
inicio_sint = caso_tmp["FECHA_SINTOMAS"].min()
deceso = caso_tmp["FECHA_DEF"].min()
recuperacion = inicio_sint+pd.Timedelta('14 days')
final = fechas[-1]

In [60]:
p = bpl.figure(x_axis_type="datetime",x_range=(inicio_sint-(final-inicio_sint)*0.1,final+(final-inicio_sint)*0.4),
               y_range=[" ","UCI","Intubado","Neumonía","Hospitalización","Resultado pruebas","Síntomas y defunción",""],
               plot_width=1000,plot_height=300,
               title="Paciente " + registro,
               toolbar_location=None)

In [61]:
p.line(x=[inicio_sint]*2,y=[["Síntomas y defunción",-0.45],["Síntomas y defunción",0.45]],color="blue",line_width=3,legend_label="Inicio síntomas")
if not pd.isnull(deceso):
    p.line(x=[deceso]*2,y=[["Síntomas y defunción",-0.45],["Síntomas y defunción",0.45]],color="black",line_width=3,legend_label="Defunción")
    p.line(x=[inicio_sint,deceso],y=[["Síntomas y defunción"],["Síntomas y defunción"]],color="black",line_width=0.8)
else:
    deceso = final
    p.line(x=[inicio_sint,final],y=[["Síntomas y defunción"],["Síntomas y defunción"]],color="black",line_width=0.8)
if (recuperacion <= final)&(recuperacion <= deceso):
    p.line(x=[inicio_sint+pd.Timedelta('14 days')]*2,y=[["Síntomas y defunción",-0.45],["Síntomas y defunción",0.45]],line_dash="dotted",color="purple",line_width=3,legend_label="14 días")

AttributeError: unexpected attribute 'legend_label' to Line, possible attributes are js_event_callbacks, js_property_callbacks, line_alpha, line_cap, line_color, line_dash, line_dash_offset, line_join, line_width, name, subscribed_events, tags, x or y

In [62]:
for valor,color,label in zip([3,2,1],[pendiente,no_positivo,positivo],["Prueba Pendiente","Prueba No positiva","Prueba Positiva"]):
    data_temporal = caso_tmp[(caso_tmp["RESULTADO"]==valor)&(caso_tmp["FECHA_ARCHIVO"]<=final)]
    if len(data_temporal)>0:
        p.rect(x=data_temporal["FECHA_ARCHIVO"],y=["Resultado pruebas"]*len(data_temporal),height=0.9,width=fechas[1]-fechas[0],color=color,line_color="white",legend_label=label)

AttributeError: unexpected attribute 'legend_label' to Rect, possible attributes are angle, angle_units, dilate, fill_alpha, fill_color, height, height_units, js_event_callbacks, js_property_callbacks, line_alpha, line_cap, line_color, line_dash, line_dash_offset, line_join, line_width, name, subscribed_events, tags, width, width_units, x or y

In [63]:
for valor,color,label in zip([1,2,99],[ambulatorio, hospitalizado,no_esp],["Ambulatorio","Hospitalizado","No especificado"]):
    data_temporal = caso_tmp[(caso_tmp["TIPO_PACIENTE"]==valor)&(caso_tmp["FECHA_ARCHIVO"]<=final)]
    if len(data_temporal)>0:
        p.rect(x=data_temporal["FECHA_ARCHIVO"],y=["Hospitalización"]*len(data_temporal),height=0.9,width=fechas[1]-fechas[0],color=color,line_color="white",legend_label=label)

AttributeError: unexpected attribute 'legend_label' to Rect, possible attributes are angle, angle_units, dilate, fill_alpha, fill_color, height, height_units, js_event_callbacks, js_property_callbacks, line_alpha, line_cap, line_color, line_dash, line_dash_offset, line_join, line_width, name, subscribed_events, tags, width, width_units, x or y

In [64]:
for valor,color,label in zip([97,98,99,2,1],[no_aplica, se_ignora, no_esp, no, si],["No aplica","Se ignora","No especificado","No","Si"]):
    data_temporal = caso_tmp[(caso_tmp["INTUBADO"]==valor)&(caso_tmp["FECHA_ARCHIVO"]<=final)]
    if len(data_temporal)>0:
        p.rect(x=data_temporal["FECHA_ARCHIVO"],y=["Intubado"]*len(data_temporal),height=0.9,width=fechas[1]-fechas[0],color=color,line_color="white",legend_label=label)

AttributeError: unexpected attribute 'legend_label' to Rect, possible attributes are angle, angle_units, dilate, fill_alpha, fill_color, height, height_units, js_event_callbacks, js_property_callbacks, line_alpha, line_cap, line_color, line_dash, line_dash_offset, line_join, line_width, name, subscribed_events, tags, width, width_units, x or y

In [65]:
for valor,color,label in zip([97,98,99,2,1],[no_aplica, se_ignora, no_esp, no, si],["No aplica","Se ignora","No especificado","No","Si"]):
    data_temporal = caso_tmp[(caso_tmp["NEUMONIA"]==valor)&(caso_tmp["FECHA_ARCHIVO"]<=final)]
    if len(data_temporal)>0:
        p.rect(x=data_temporal["FECHA_ARCHIVO"],y=["Neumonía"]*len(data_temporal),height=0.9,width=fechas[1]-fechas[0],color=color,line_color="white",legend_label=label)

AttributeError: unexpected attribute 'legend_label' to Rect, possible attributes are angle, angle_units, dilate, fill_alpha, fill_color, height, height_units, js_event_callbacks, js_property_callbacks, line_alpha, line_cap, line_color, line_dash, line_dash_offset, line_join, line_width, name, subscribed_events, tags, width, width_units, x or y

In [66]:
for valor,color in zip([97,98,99,2,1],[no_aplica, se_ignora, no_esp, no, si]):
    data_temporal = caso_tmp[(caso_tmp["UCI"]==valor)&(caso_tmp["FECHA_ARCHIVO"]<=final)]
    p.rect(x=data_temporal["FECHA_ARCHIVO"],y=["UCI"]*len(data_temporal),height=0.9,width=fechas[1]-fechas[0],color=color,line_color="white")

In [67]:
bpl.show(p)

In [1096]:
export_png(p, filename=FIGURAS.joinpath(f"{registro}_{fecha_file}.png"))

'E:\\proyectos\\morlan\\covid_plots\\graficas\\167f53_12_05_2020.png'